In [22]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd
import numpy as np


driver = webdriver.Chrome()
driver.maximize_window()
driver.get('https://www.redbus.in/online-booking/ktcl/?utm_source=rtchometile');
time.sleep(3)

scrolling = True
while scrolling:
    old_page = driver.page_source
    body = driver.find_element(By.TAG_NAME,"body")
    body.send_keys(Keys.PAGE_DOWN)

    time.sleep(2)
    new_page = driver.page_source

    if new_page == old_page:
        scrolling = False

page1 = driver.find_elements(By.CSS_SELECTOR,"a[class='route']")
route1 = [i.text for i in page1]
link1 = [i.get_attribute('href') for i in page1]
ktcl1 = pd.DataFrame({'route':route1,'route link':link1})

time.sleep(3)
driver.find_element(By.XPATH,"//div[contains(text(),'2')]").click()
page2 = driver.find_elements(By.CSS_SELECTOR,"a[class='route']")
route2 = [i.text for i in page2]
link2 = [i.get_attribute('href') for i in page2]
ktcl2 = pd.DataFrame({'route':route2,'route link':link2})

time.sleep(5)
driver.find_element(By.XPATH,"//div[contains(text(),'3')]").click()
page3 = driver.find_elements(By.CSS_SELECTOR,"a[class='route']")
route3 = [i.text for i in page3]
link3 = [i.get_attribute('href') for i in page3]
ktcl3 = pd.DataFrame({'route':route3,'route link':link3})


time.sleep(5)
driver.find_element(By.XPATH,"//div[contains(text(),'4')]").click()
page4 = driver.find_elements(By.CSS_SELECTOR,'a[class="route"]')
route4 = [i.text for i in page4]
link4 = [i.get_attribute('href') for i in page4]
ktcl4 = pd.DataFrame({'route':route4,'route link':link4})

ktcl = pd.concat([ktcl1,ktcl2,ktcl3,ktcl4])
ktcl = ktcl.reset_index()
ktcl.drop('index',axis = 1, inplace = True)

ktcl_url = ktcl['route link']


from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd
import numpy as np

# Initialize the web driver
driver = webdriver.Chrome()
driver.maximize_window()

# Base DataFrame to collect all bus information
all_bus_data = pd.DataFrame()

# Loop through each URL
for url in ktcl['route link']:
    driver.get(url)
    time.sleep(10)

    # Scroll to the bottom of the page to load all data
    last_height = driver.execute_script("return document.body.scrollHeight")

    scrolling = True
    while scrolling:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)  # Adjust this delay based on page load speed
        new_height = driver.execute_script("return document.body.scrollHeight")
        
        if new_height == last_height:
            scrolling = False
        else:
            last_height = new_height

    # Extract data
    bus_route = driver.find_element(By.CSS_SELECTOR,'h1[class="D136_h1"]')
    bus_name = driver.find_elements(By.XPATH, '//div[@class="travels lh-24 f-bold d-color"]')
    bus_type = driver.find_elements(By.XPATH, '//div[@class="bus-type f-12 m-top-16 l-color evBus"]')
    departure_time = driver.find_elements(By.XPATH, '//div[@class="dp-time f-19 d-color f-bold"]')
    duration = driver.find_elements(By.XPATH, '//div[@class="dur l-color lh-24"]')
    arrival_time = driver.find_elements(By.XPATH, '//div[@class="bp-time f-19 d-color disp-Inline"]')
    star_rating = driver.find_elements(By.XPATH, '//div[@class="rating-sec lh-24"]')
    price = driver.find_elements(By.XPATH, '//span[@class="f-19 f-bold"]')
    seats_available = driver.find_elements(By.XPATH, '//div[@class="seat-left m-top-30"]')

    # Convert elements to lists
    bus_route_name = bus_route.text
    bus_name_lst = [i.text for i in bus_name]
    bus_type_lst = [i.text for i in bus_type]
    departure_time_lst = [i.text for i in departure_time]
    duration_lst = [i.text for i in duration]
    arrival_time_lst = [i.text for i in arrival_time]
    star_rating_lst = [i.text for i in star_rating]
    price_lst = [i.text for i in price]
    seats_available_lst = []
    for seat in seats_available:
        text = seat.text  # Get the text content
        number_of_seats = text.split()[0]  # Split and take the first part, which is the number
        seats_available_lst.append(int(number_of_seats)) 

    # Pad each list with NaN values
    max_length = max(len(bus_name_lst), len(bus_type_lst), len(departure_time_lst),
                     len(duration_lst), len(arrival_time_lst), len(star_rating_lst),
                     len(price_lst), len(seats_available_lst))

    bus_name_lst += [np.nan] * (max_length - len(bus_name_lst))
    bus_type_lst += [np.nan] * (max_length - len(bus_type_lst))
    departure_time_lst += [np.nan] * (max_length - len(departure_time_lst))
    duration_lst += [np.nan] * (max_length - len(duration_lst))
    arrival_time_lst += [np.nan] * (max_length - len(arrival_time_lst))
    star_rating_lst += [np.nan] * (max_length - len(star_rating_lst))
    price_lst += [np.nan] * (max_length - len(price_lst))
    seats_available_lst += [np.nan] * (max_length - len(seats_available_lst))

    # Create a DataFrame for the current URL
    ktcl_bus = pd.DataFrame({
        'Route Name': [bus_route_name] * max_length,
        'Route Link': [url] * max_length,
        'Bus Name': bus_name_lst,
        'Bus Type': bus_type_lst,
        'Departure Time': departure_time_lst,
        'Duration': duration_lst,
        'Arrival Time': arrival_time_lst,
        'Star Rating': star_rating_lst,
        'Price': price_lst,
        'Seats Available': seats_available_lst
    })

    # Append the current DataFrame to the all_bus_data DataFrame
    all_bus_data = pd.concat([all_bus_data, ktcl_bus], ignore_index=True)



# Convert 'Price' and 'Star Rating' to numeric, forcing non-numeric values to NaN
all_bus_data['Price'] = pd.to_numeric(all_bus_data['Price'], errors='coerce')
all_bus_data['Star Rating'] = pd.to_numeric(all_bus_data['Star Rating'], errors='coerce')

# Fill NaNs in these columns with their mean, rounded to the nearest integer
all_bus_data['Price'] = all_bus_data['Price'].fillna(all_bus_data['Price'].mean().round(0).astype(int))
all_bus_data['Star Rating'] = all_bus_data['Star Rating'].fillna(all_bus_data['Star Rating'].mean().round(0).astype(int))
all_bus_data['Seats Available']=all_bus_data['Seats Available'].fillna(all_bus_data['Seats Available'].mean().round(0).astype(int))


# Assign to 'a' after filling nulls
ktcl_bus = all_bus_data
ktcl_bus





,Route Name,Route Link,Bus Name,Bus Type,Departure Time,Duration,Arrival Time,Star Rating,Price,Seats Available
0,Pune to Goa Bus,https://www.redbus.in/bus-tickets/pune-to-goa,Atmaram Gobus,VE A/C Sleeper (2+1),21:00,10h 30m,07:30,4.7,1499.0,3.0
1,Pune to Goa Bus,https://www.redbus.in/bus-tickets/pune-to-goa,Atmaram Gobus,VE A/C Sleeper (2+1),22:30,10h 30m,09:00,4.7,1499.0,7.0
2,Pune to Goa Bus,https://www.redbus.in/bus-tickets/pune-to-goa,IntrCity SmartBus,Bharat Benz A/C Seater /Sleeper (2+1),21:15,10h 50m,08:05,4.5,801.0,19.0
3,Pune to Goa Bus,https://www.redbus.in/bus-tickets/pune-to-goa,AdAnand Tours And Travels,Volvo Multi-Axle A/C Sleeper (2+1),22:30,10h 00m,08:30,3.8,1599.0,9.0
4,Pune to Goa Bus,https://www.redbus.in/bus-tickets/pune-to-goa,Ashray Travels,Bharat Benz A/C Sleeper (2+1),21:05,11h 25m,08:30,4.3,1599.0,18.0
...,...,...,...,...,...,...,...,...,...,...
427,Shivamogga to Goa Bus,https://www.redbus.in/bus-tickets/shimoga-to-goa,Kadamba Transport Corporation Limited (KTCL) -...,Non AC Seater (2+2),23:00,09h 00m,08:00,2.3,450.0,38.0
428,Shivamogga to Goa Bus,https://www.redbus.in/bus-tickets/shimoga-to-goa,Sea Bird Tourist,NON A/C Sleeper (2+1),23:45,08h 15m,08:00,3.1,900.0,33.0
429,Goa to Shivamogga Bus,https://www.redbus.in/bus-tickets/goa-to-shimoga,Kadamba Transport Corporation Limited (KTCL) -...,Non AC Seater 2+2,15:00,10h 03m,01:03,1.7,450.0,37.0
430,Goa to Shivamogga Bus,https://www.redbus.in/bus-tickets/goa-to-shimoga,Sea Bird Tourist,NON A/C Sleeper (2+1),17:30,09h 00m,02:30,4.0,900.0,34.0


In [ ]:
ktcl_bus.to_csv("ktcl_bus.csv", index = False)


In [24]:
ktcl_bus.isnull().sum()

Route Name         0
Route Link         0
Bus Name           0
Bus Type           0
Departure Time     0
Duration           0
Arrival Time       0
Star Rating        0
Price              0
Seats Available    0
dtype: int64

In [15]:
a.isnull().sum()

Route Name         0
Route Link         0
Bus Name           0
Bus Type           0
Departure Time     0
Duration           0
Arrival Time       0
Star Rating        0
Price              0
Seats Available    0
dtype: int64

In [16]:
a

,Route Name,Route Link,Bus Name,Bus Type,Departure Time,Duration,Arrival Time,Star Rating,Price,Seats Available
0,Pune to Goa Bus,https://www.redbus.in/bus-tickets/pune-to-goa,Atmaram Gobus,VE A/C Sleeper (2+1),21:00,10h 45m,07:45,4.7,1499.0,3.0
1,Pune to Goa Bus,https://www.redbus.in/bus-tickets/pune-to-goa,Atmaram Gobus,VE A/C Sleeper (2+1),22:30,10h 15m,08:45,4.7,1499.0,7.0
2,Pune to Goa Bus,https://www.redbus.in/bus-tickets/pune-to-goa,IntrCity SmartBus,Bharat Benz A/C Seater /Sleeper (2+1),21:15,10h 50m,08:05,4.5,801.0,19.0
3,Pune to Goa Bus,https://www.redbus.in/bus-tickets/pune-to-goa,AdAnand Tours And Travels,Volvo Multi-Axle A/C Sleeper (2+1),22:30,10h 00m,08:30,3.8,1599.0,9.0
4,Pune to Goa Bus,https://www.redbus.in/bus-tickets/pune-to-goa,Ashray Travels,Bharat Benz A/C Sleeper (2+1),21:05,11h 25m,08:30,4.3,1599.0,19.0
...,...,...,...,...,...,...,...,...,...,...
461,Belagavi to Goa Bus,https://www.redbus.in/bus-tickets/belagavi-to-goa,PVS Tours and Travels,A/C Sleeper (2+1),11:05,05h 00m,16:05,3.3,999.0,22.0
462,Belagavi to Goa Bus,https://www.redbus.in/bus-tickets/belagavi-to-goa,Humsafar Travels,Bharat Benz A/C Sleeper (2+1),03:30,04h 30m,08:00,1.2,402.0,22.0
463,Belagavi to Goa Bus,https://www.redbus.in/bus-tickets/belagavi-to-goa,Shirdi Xpress,A/C Sleeper (2+1),08:15,03h 15m,11:30,1.2,1000.0,22.0
464,Belagavi to Goa Bus,https://www.redbus.in/bus-tickets/belagavi-to-goa,PVS Tours and Travels,A/C Sleeper (2+1),10:35,04h 25m,15:00,1.0,799.0,22.0


In [9]:

# convert to csv file
all_ktcl_buses_withnul.to_csv('ktcl_bus_withnull.csv', index=False)

In [3]:
all_bus_data_ktcl = all_bus_data.fillna("Unavailable")

In [4]:
all_bus_data_ktcl

,Route Name,Route Link,Bus Name,Bus Type,Departure Time,Duration,Arrival Time,Star Rating,Price,Seats Available
0,Pune to Goa Bus,https://www.redbus.in/bus-tickets/pune-to-goa,Kadamba Transport Corporation Limited (KTCL) -...,A/C Sleeper (2+1),19:15,10h 35m,05:50,4.7,1100,7.0
1,Pune to Goa Bus,https://www.redbus.in/bus-tickets/pune-to-goa,Atmaram Gobus,VE A/C Sleeper (2+1),21:00,10h 30m,07:30,4.7,900,16.0
2,Pune to Goa Bus,https://www.redbus.in/bus-tickets/pune-to-goa,Atmaram Gobus,VE A/C Sleeper (2+1),22:30,10h 15m,08:45,4.4,900,23.0
3,Pune to Goa Bus,https://www.redbus.in/bus-tickets/pune-to-goa,Ashray Travels,Bharat Benz A/C Sleeper (2+1),21:05,11h 25m,08:30,4.4,900,23.0
4,Pune to Goa Bus,https://www.redbus.in/bus-tickets/pune-to-goa,Ashray Travels,Non A/C Seater / Sleeper (2+1),21:20,10h 25m,07:45,4.4,500,35.0
...,...,...,...,...,...,...,...,...,...,...
588,Belagavi to Goa Bus,https://www.redbus.in/bus-tickets/belagavi-to-goa,PVS Tours and Travels,A/C Sleeper (2+1),07:30,04h 00m,11:30,3.3,900,Unavailable
589,Belagavi to Goa Bus,https://www.redbus.in/bus-tickets/belagavi-to-goa,PVS Tours and Travels,A/C Sleeper (2+1),11:05,03h 55m,15:00,3.3,1397,Unavailable
590,Belagavi to Goa Bus,https://www.redbus.in/bus-tickets/belagavi-to-goa,Humsafar Travels,Bharat Benz A/C Sleeper (2+1),03:30,04h 30m,08:00,Unavailable,1250,Unavailable
591,Belagavi to Goa Bus,https://www.redbus.in/bus-tickets/belagavi-to-goa,Shirdi Xpress,A/C Sleeper (2+1),08:15,03h 15m,11:30,Unavailable,419,Unavailable


In [6]:
all_bus_data_ktcl.isnull().sum()

Route Name         0
Route Link         0
Bus Name           0
Bus Type           0
Departure Time     0
Duration           0
Arrival Time       0
Star Rating        0
Price              0
Seats Available    0
dtype: int64

In [5]:
all_bus_data.isnull().mean()*100

Route Name          0.000000
Route Link          0.000000
Bus Name            0.000000
Bus Type            0.000000
Departure Time      0.000000
Duration            0.000000
Arrival Time        0.000000
Star Rating        56.692913
Price              10.761155
Seats Available     7.086614
dtype: float64

In [12]:
new_all_bus_data = all_bus_data.dropna()

In [19]:
new_all_bus_data.columns

Index(['Route Name', 'Route Link', 'Bus Name', 'Bus Type', 'Departure Time',
       'Duration', 'Arrival Time', 'Star Rating', 'Price', 'Seats Available'],
      dtype='object')

In [13]:
new_all_bus_data

,Route Name,Route Link,Bus Name,Bus Type,Departure Time,Duration,Arrival Time,Star Rating,Price,Seats Available
0,Pune to Goa Bus,https://www.redbus.in/bus-tickets/pune-to-goa,Kadamba Transport Corporation Limited (KTCL) -...,A/C Sleeper (2+1),19:15,10h 35m,05:50,4.3,1400,3.0
1,Pune to Goa Bus,https://www.redbus.in/bus-tickets/pune-to-goa,Ashray Travels,Non A/C Seater / Sleeper (2+1),21:20,10h 25m,07:45,4.2,500,10.0
2,Pune to Goa Bus,https://www.redbus.in/bus-tickets/pune-to-goa,Zingbus Plus,VE A/C Sleeper (2+1),00:10,12h 55m,13:05,4.2,1420,20.0
3,Pune to Goa Bus,https://www.redbus.in/bus-tickets/pune-to-goa,Prasanna - Purple Bus,VE A/C Sleeper (2+1),22:30,11h 00m,09:30,4.4,1350,9.0
5,Goa to Pune Bus,https://www.redbus.in/bus-tickets/goa-to-pune,Kadamba Transport Corporation Limited (KTCL) -...,A/C Sleeper (2+1),19:00,12h 00m,07:00,4.6,900,16.0
...,...,...,...,...,...,...,...,...,...,...
348,Goa to Satara Bus,https://www.redbus.in/bus-tickets/goa-to-satara,Atmaram Travels ( Gogte ),NON A/C Sleeper (2+1),21:15,08h 30m,05:45,4.0,500,26.0
349,Goa to Satara Bus,https://www.redbus.in/bus-tickets/goa-to-satara,Atmaram Travels (Shreyas),NON A/C Sleeper (2+1),20:30,08h 30m,05:00,4.0,920,30.0
350,Goa to Satara Bus,https://www.redbus.in/bus-tickets/goa-to-satara,Laxmi-Princess Nova,Non A/C Seater / Sleeper (2+1),19:00,09h 00m,04:00,4.0,3500,12.0
351,Goa to Satara Bus,https://www.redbus.in/bus-tickets/goa-to-satara,Prasanna - Purple Bus,VE A/C Sleeper (2+1),22:15,07h 30m,05:45,4.0,1050,25.0


In [22]:
import mysql.connector as db

# Step 1: Connect to MySQL without specifying a database (for creating schema)
db_connection = db.connect(
    host='localhost',
    user='root',
    password='Ram@1496'  # Your MySQL root password
)

cursor = db_connection.cursor()

# Step 2: Create the schema (database)
cursor.execute("CREATE DATABASE IF NOT EXISTS ktcl")
cursor.execute("USE ktcl")  # Switch to the new database

# Step 3: Create the table with id as the Primary Key and Auto-Increment
table_creation_query = """
CREATE TABLE IF NOT EXISTS bus_data (
    id INT AUTO_INCREMENT PRIMARY KEY,
    route_name VARCHAR(255),
    route_link VARCHAR(255),
    bus_name VARCHAR(255),
    bus_type VARCHAR(100),
    departure_time VARCHAR(50),
    duration VARCHAR(50),
    arrival_time VARCHAR(50),
    star_rating FLOAT,
    price DECIMAL(10, 2),
    seats_available INT
)
"""
cursor.execute(table_creation_query)

# Step 4: Insert data from the DataFrame into the MySQL table
insert_query = """
INSERT INTO bus_data (route_name, route_link, bus_name, bus_type, departure_time, 
                      duration, arrival_time, star_rating, price, seats_available)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

# Assuming 'new_all_bus_data' is your DataFrame
# Loop through each row in the DataFrame and insert the values
for index, row in new_all_bus_data.iterrows():
    cursor.execute(insert_query, (
        row['Route Name'], row['Route Link'], row['Bus Name'], row['Bus Type'],
        row['Departure Time'], row['Duration'], row['Arrival Time'],
        row['Star Rating'], row['Price'], row['Seats Available']
    ))

# Step 5: Commit the transaction to save the data
db_connection.commit()

# Step 6: Close the cursor and connection
cursor.close()
db_connection.close()

print("Data inserted into MySQL successfully!")

Data inserted into MySQL successfully!
